In [3]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [4]:
# with open("options.txt", 'r') as f:
#     options = f.readlines()
#     options = {option.split("=")[0]: option.split("=")[1].strip() for option in options}
# print(options)

In [5]:
RUG = pd.read_pickle("temp.pkl")
RUG = RUG.filter(['Oosterbeek-hoog\Fletcher BV|Supplied'])
RUG=RUG.head(int(525600/2))
RUG

,Oosterbeek-hoog\Fletcher BV|Supplied
date,
2017-01-01 00:00:00+01:00,1.03
2017-01-01 00:01:00+01:00,1.03
2017-01-01 00:02:00+01:00,1.03
2017-01-01 00:03:00+01:00,1.03
2017-01-01 00:04:00+01:00,1.03
...,...
2017-07-02 11:55:00+01:00,0.42
2017-07-02 11:56:00+01:00,0.42
2017-07-02 11:57:00+01:00,0.42


In [6]:
# fix random seed for reproducibility
# tf.random.set_seed(7)

In [7]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(RUG)

In [8]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

176076 86724


In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [11]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [12]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [13]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [14]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=8, batch_size=1, verbose=2)

Epoch 1/8
176074/176074 - 497s - loss: 1.7228e-04 - accuracy: 6.7585e-04 - 497s/epoch - 3ms/step
Epoch 2/8
176074/176074 - 503s - loss: 1.0431e-04 - accuracy: 6.8153e-04 - 503s/epoch - 3ms/step
Epoch 3/8
176074/176074 - 490s - loss: 1.0103e-04 - accuracy: 6.8153e-04 - 490s/epoch - 3ms/step
Epoch 4/8
176074/176074 - 455s - loss: 9.9793e-05 - accuracy: 6.8153e-04 - 455s/epoch - 3ms/step
Epoch 5/8
176074/176074 - 478s - loss: 9.8783e-05 - accuracy: 6.8153e-04 - 478s/epoch - 3ms/step
Epoch 6/8
176074/176074 - 479s - loss: 9.8159e-05 - accuracy: 6.8153e-04 - 479s/epoch - 3ms/step
Epoch 7/8
176074/176074 - 479s - loss: 9.7834e-05 - accuracy: 6.8153e-04 - 479s/epoch - 3ms/step
Epoch 8/8
176074/176074 - 475s - loss: 9.7687e-05 - accuracy: 6.8153e-04 - 475s/epoch - 3ms/step


In [15]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

2711/2711 [==============================] - 3s 1ms/step
Train Score: 0.05 RMSE
Test Score: 0.06 RMSE
